https://github.com/ggerganov/llama.cpp

In [ ]:
!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 6.6 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git pull && make clean && LLAMA_CUBLAS=1 make
!pip install -r llama.cpp/requirements.txt

Cloning into 'llama.cpp'...
remote: Enumerating objects: 9422, done.
remote: Counting objects: 100% (4131/4131), done.
remote: Compressing objects: 100% (317/317), done.
remote: Total 9422 (delta 3981), reused 3829 (delta 3814), pack-reused 5291
Receiving objects: 100% (9422/9422), 8.54 MiB | 30.14 MiB/s, done.
Resolving deltas: 100% (6584/6584), done.
Already up to date.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_K_QUANTS  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -Wdouble-promotion -pthread -march=native -mtune=native 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_K_QUANTS  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declaration

In [ ]:
!pip install --quiet accelerate==0.21.0
!pip install --quiet peft==0.4.0
!pip install --quiet bitsandbytes==0.40.2
!pip install --quiet transformers==4.31.0
!pip install --quiet trl==0.4.7
!pip install --quiet wandb
!pip install --quiet evaluate
!pip install --quiet xformers

In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import time
import numpy as np

In [ ]:
MODEL_ID = "foobar8675/llama-2-7b-sentiment-classifier"
MODEL_NAME = MODEL_ID.split('/')[-1]
print(MODEL_NAME)

llama-2-7b-sentiment-classifier


In [ ]:
# Download model
!git lfs install
!git clone https://huggingface.co/{MODEL_ID} models/7b

Git LFS initialized.
Cloning into 'models/7b'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 5), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (23/23), 480.63 KiB | 2.20 MiB/s, done.
Filtering content: 100% (2/2), 4.55 GiB | 18.18 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


In [ ]:
!git lfs install
!git clone https://huggingface.co/decapoda-research/llama-7b-hf vocab

Git LFS initialized.
Cloning into 'vocab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 91 (delta 6), reused 91 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), 22.58 KiB | 1.33 MiB/s, done.
Filtering content: 100% (34/34), 12.55 GiB | 145.99 MiB/s, done.


In [ ]:
# convert the 7B model to ggml FP16 format
!python llama.cpp/convert.py models/7b/ --vocab-dir vocab

Loading model file models/7b/pytorch_model-00001-of-00002.bin
Loading model file models/7b/pytorch_model-00001-of-00002.bin
Loading model file models/7b/pytorch_model-00002-of-00002.bin
params = Params(n_vocab=32000, n_embd=4096, n_layer=32, n_ctx=4096, n_ff=11008, n_head=32, n_head_kv=32, f_norm_eps=1e-05, f_rope_freq_base=None, f_rope_scale=None, ftype=None, path_model=PosixPath('models/7b'))
Loading vocab file 'vocab/tokenizer.model', type 'spm'
Permuting layer 0
Permuting layer 1
Permuting layer 2
Permuting layer 3
Permuting layer 4
Permuting layer 5
Permuting layer 6
Permuting layer 7
Permuting layer 8
Permuting layer 9
Permuting layer 10
Permuting layer 11
Permuting layer 12
Permuting layer 13
Permuting layer 14
Permuting layer 15
Permuting layer 16
Permuting layer 17
Permuting layer 18
Permuting layer 19
Permuting layer 20
Permuting layer 21
Permuting layer 22
Permuting layer 23
Permuting layer 24
Permuting layer 25
Permuting layer 26
Permuting layer 27
Permuting layer 28
Permut

In [ ]:
# quantize the model to 4-bits (using q4_0 method)
!./llama.cpp/quantize ./models/7b/ggml-model-f16.gguf ./models/7b/ggml-model-q4_0.gguf q4_0

ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
main: build = 1299 (f5ef5cf)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing './models/7b/ggml-model-f16.gguf' to './models/7b/ggml-model-q4_0.gguf' as Q4_0
llama_model_loader: loaded meta data with 18 key-value pairs and 291 tensors from ./models/7b/ggml-model-f16.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight f16      [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight f16      [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight f16      [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight f16      [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight f16      [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor 

In [ ]:
# update the gguf filetype to current if older version is unsupported by another application
!./llama.cpp/quantize ./models/7b/ggml-model-q4_0.gguf ./models/7b/ggml-model-q4_0-v2.gguf COPY

ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
main: build = 1299 (f5ef5cf)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing './models/7b/ggml-model-q4_0.gguf' to './models/7b/ggml-model-q4_0-v2.gguf' as COPY
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models/7b/ggml-model-q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - te

In [ ]:
# run the inference
!./llama.cpp/main -m ./models/7b/ggml-model-q4_0.gguf -n 128

Log start
main: build = 1299 (f5ef5cf)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1696123724
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models/7b/ggml-model-q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_0     [  4096, 1100

In [ ]:
from huggingface_hub import notebook_login, create_repo, HfApi
username = "foobar8675"
api = HfApi()

# Create repo
create_repo(
    repo_id=f"foobar8675/{MODEL_NAME}-GGML",
    repo_type="model",
    exist_ok=True
)

# Upload bin models
api.upload_folder(
    folder_path="models/7b",
    repo_id=f"{username}/{MODEL_NAME}-GGML",
    allow_patterns=f"*ggml-model-q4_0.gguf*",
)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

ggml-model-q4_0.gguf:   0%|          | 0.00/3.83G [00:00<?, ?B/s]

ggml-model-f16.gguf:   0%|          | 0.00/13.5G [00:00<?, ?B/s]

ggml-model-q4_0.gguf:   0%|          | 0.00/3.83G [00:00<?, ?B/s]

'https://huggingface.co/foobar8675/llama-2-7b-sentiment-classifier-GGML/tree/main/'